Workshop activity by SMU Quantum Computing Society
Resources from Prof Paul Griffin, IBM Qiskit.

In [1]:
from IPython.display import Image
Image(url='https://i.postimg.cc/9frQkjds/IMG-9753.jpg')

# Portfolio Optimization


## Introduction

This tutorial shows how to solve the following mean-variance portfolio optimization problem for $n$ assets:

$$\begin{aligned}
\min_{x \in \{0, 1\}^n}  q x^T \Sigma x - \mu^T x\\
\text{subject to: } 1^T x = B
\end{aligned}$$

where we use the following notation:

- $x \in \{0, 1\}^n$ denotes the vector of binary decision variables, which indicate which assets to pick ($x[i] = 1$) and which not to pick ($x[i] = 0$),
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- and $B$ denotes the budget, i.e. the number of assets to be selected out of $n$.

We assume the following simplifications:
- all assets have the same price (normalized to 1),
- the full budget $B$ has to be spent, i.e. one has to select exactly $B$ assets.

The equality constraint $1^T x = B$ is mapped to a penalty term $(1^T x - B)^2$ which is scaled by a parameter and subtracted from the objective function.
The resulting problem can be mapped to a Hamiltonian whose ground state corresponds to  the optimal solution.
This notebook shows how to use the Sampling Variational Quantum Eigensolver (`SamplingVQE`) or the Quantum Approximate Optimization Algorithm (`QAOA`) from [Qiskit Algorithms](https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.html#minimum-eigensolvers) to find the optimal solution for a given set of parameters.

Experiments on real quantum hardware for this problem are reported for instance in the following paper:
<br>
[Improving Variational Quantum Optimization using CVaR. Barkoutsos et al. 2019.](https://arxiv.org/abs/1907.04769)

In [ ]:
#!pip install seaborn
!pip install -r requirements.txt

from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Define problem instance

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian translated from the portfolio problem. We use a random portfolio problem for this notebook.

In [ ]:
# set number of assets (= number of qubits)
num_assets = 4
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]

data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()

mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

print(data)
print(stocks)
print(mu)
print(sigma)

## Visualise Sigma (covariance)

In [ ]:
#plt.imshow(sigma, interpolation="nearest")
#plt.show()

c = plt.imshow(sigma, cmap ='Greens',
                 vmax = 0.0005,
                 extent =[0, 3, 0, 3],
                 interpolation ='nearest',
                 origin ='lower')

#plt.style.use('seaborn-white')
#plt.style.use("seaborn-v0_8-white")
plt.style.use("seaborn-v0_8-white")

plt.colorbar(c)
plt.show()

## Setting Risk Factor (q)

In [ ]:
q=0
#q = 0.5  # set risk factor
#q = 10  # set risk factor

budget = num_assets // 2  # set budget
penalty = num_assets  # set parameter to scale the budget penalty term

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

We define some utility methods to print the results in a nice format.

In [ ]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

## NumPyMinimumEigensolver (as a classical reference)
Lets solve the problem. First classically...

We can now use the Operator we built above without regard to the specifics of how it was created. We set the algorithm for the NumPyMinimumEigensolver so we can have a classical reference. The problem is set for 'ising'. Backend is not required since this is computed classically not using quantum computation. The result is returned as a dictionary.

In [ ]:
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)

result = exact_eigensolver.solve(qp)

print_result(result)

## Solution using `SamplingVQE`
We can now use the Sampling Variational Quantum Eigensolver (`SamplingVQE`) to solve the problem. We will specify the optimizer and variational form to be used.

In [ ]:
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorSampler
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
sampler = StatevectorSampler()
svqe_mes = SamplingVQE(sampler=sampler, ansatz=ry, optimizer=cobyla)
svqe = MinimumEigenOptimizer(svqe_mes)

result = svqe.solve(qp)

# Get the result details
eigenstate = result.min_eigen_solver_result.eigenstate

# Handle different eigenstate formats
if isinstance(eigenstate, QuasiDistribution):
    probabilities = eigenstate.binary_probabilities()
elif isinstance(eigenstate, dict):
    probabilities = eigenstate
else:
    probabilities = {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}

# Print as table
print("Optimal: selection [{}], value {:.4f}".format(
    ", ".join(map(str, result.x.astype(int))), result.fval))
print("\n----------------- Full result ---------------------")
print("selection\t\tvalue\t\tprobability")
print("---------------------------------------------------")

# Sort by probability (highest first)
sorted_probs = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

for bitstring, prob in sorted_probs:
    x = np.array([int(i) for i in list(reversed(bitstring))])
    value = portfolio.to_quadratic_program().objective.evaluate(x)
    print("{}\t\t{:.5f}\t\t{:.5f}".format(x, value, prob))

## Solution using `QAOA`

We also show here a result using the Quantum Approximate Optimization Algorithm (`QAOA`). This is another variational algorithm and it uses an internal variational form that is created based on the problem.

In [ ]:
from qiskit.primitives import StatevectorSampler

algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)

sampler = StatevectorSampler()

qaoa_mes = QAOA(sampler=sampler, optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

# Get the result details
eigenstate = result.min_eigen_solver_result.eigenstate

# Handle different eigenstate formats
if isinstance(eigenstate, QuasiDistribution):
    probabilities = eigenstate.binary_probabilities()
elif isinstance(eigenstate, dict):
    probabilities = eigenstate
else:
    probabilities = {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}

# Print as table
print("QAOA Result")
print("Optimal: selection [{}], value {:.4f}".format(
    ", ".join(map(str, result.x.astype(int))), result.fval))
print("\n----------------- Full result ---------------------")
print("selection\t\tvalue\t\tprobability")
print("---------------------------------------------------")

# Sort by probability (highest first)
sorted_probs = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

for bitstring, prob in sorted_probs:
    x = np.array([int(i) for i in list(reversed(bitstring))])
    value = portfolio.to_quadratic_program().objective.evaluate(x)
    print("{}\t\t{:.4f}\t\t{:.4f}".format(x, value, prob))